Title: MCQ TF FITB GENERATOR     
Author: Raikibul Hasan

**Table of contents**<a id='toc0_'></a>    
- [Preparation](#toc1_)    
    - [Install libraries](#toc1_1_1_)    
    - [Import libraries](#toc1_1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Preparation](#toc0_)
In this section, we will import the necessary libraries and dataset.

### <a id='toc1_1_1_'></a>[Install libraries](#toc0_)
***If you have installed  library once skip it otherwise make installed varibale false and run***

In [2]:
installed=True

In [3]:
def install_library():
    !pip install flashtext
    !pip install pywsd
    !pip install string
    !pip install nltk
    !pip install spacy
    !pip install git+https://github.com/boudinfl/pke.git
    !pip install matplotlib
    !python -m spacy download en_core_web_sm
    !pip install ipywidgets
if not installed:
    install_library()

### <a id='toc1_1_2_'></a>[Import libraries](#toc0_)

In [3]:
import nltk
import pke
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('popular')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
import requests
import json
import re
import random
# File system manangement
import os
import ipywidgets as widgets
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to C:\Users\Raikibul
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Raikibul
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to C:\Users\Raikibul
[nltk_data]    |     Hasan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to C:\Users\Raikibul
[nltk_data]    |     Hasan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to C:\Users\Raikibul
[nltk_data]    |     Hasan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to C:\Users\Ra

### Load/Insert Article/file
For generating MCQ you need to load text file or you can also copy any article and paste here.    </br>
</br> 
***Choose an option for Load Article***


In [4]:
loadsetting=widgets.RadioButtons(
    options=['example', 'fileupload', 'instert-text'],
    value='example', # Defaults to 'pineapple'
    #layout={'width': 'max-content'}, # If the items' names are long
    description='Choose a option for Load Article:',
    disabled=False
)
loadsetting

RadioButtons(description='Choose a option for Load Article:', options=('example', 'fileupload', 'instert-text'…

In [5]:
if loadsetting.value=='instert-text':
    
    uploadfile=widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt',
    multiple=False  # True to accept multiple files upload else False
    )  
    
elif loadsetting.value=='instert-text':
    x=6

else:
    x=9

uploadfile

FileUpload(value={}, description='Upload')

In [19]:

uploadfile=widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt',
    multiple=False  # True to accept multiple files upload else False
)      
box = widgets.VBox(loadsetting )
box

TraitError: Invalid selection: value not found

In [16]:
# defining some widgets
text = widgets.Textarea(
       value='',
       description='Article/text', )
box = widgets.VBox([text ])
box